In [1]:
%pylab inline
import GPy
import GPyOpt
import numpy as np
import matplotlib.pyplot as plt
from GPyOpt.util.pareto import ParetoFront
from GPyOpt.experiment_design import initial_design
from GPyOpt.acquisitions.HvEI import AcquisitionHvEI
from GPyOpt.methods import ModularBayesianMultiobjectiveOptimization

Populating the interactive namespace from numpy and matplotlib


In [2]:
def binhKorn(x,y):
    f1 = 4.*x*x + 4*y*y
    f2 = (x-5.)**2 + (y-5.)**2
    return np.array([f1,f2])

def disc_constraint(x,y):
    return (25.-(x-5)**2-y**2)

def line_constraint(x,y):
    return ((x-8)**2 + (y+3)**2 - 7.7)

In [3]:
np.set_printoptions(precision=6)

objective = None 

space =[{'name': 'var_1', 'type': 'continuous', 'domain': (0.0,5.0)},
        {'name': 'var_2', 'type': 'continuous', 'domain': (0.0,3.0)}]
space = GPyOpt.Design_space(space = space)

kernel   = None 
kernel_c = None

model_f1 = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False)
model_f2 = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False)
model_f  = [model_f1,model_f2]

model_c1 = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False)
model_c2 = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False)
model_c  = [model_c1,model_c2]

aquisition_optimizer = GPyOpt.optimization.AcquisitionOptimizer(space)

num_init = 23
typ_init = 'latin'
X_init = initial_design(typ_init,space,num_init)

Y_init = np.zeros((X_init.shape[0],2))
C_init = np.zeros((X_init.shape[0],2))

for i in range(num_init):
    Y_init[i,0],Y_init[i,1] = binhKorn(X_init[i,0],X_init[i,1])
    C_init[i,0] = disc_constraint(X_init[i,0],X_init[i,1])
    C_init[i,1] = line_constraint(X_init[i,0],X_init[i,1])

np.set_printoptions(precision=4)
print(np.hstack((X_init,Y_init,C_init)))

[[ 3.5870e+00  1.5000e+00  6.0465e+01  1.4247e+01  2.0753e+01  3.2025e+01]
 [ 3.8043e+00  2.6739e+00  8.6491e+01  6.8403e+00  1.6421e+01  4.2097e+01]
 [ 3.3696e+00  4.5652e-01  4.6250e+01  2.3302e+01  2.2133e+01  2.5688e+01]
 [ 4.0217e+00  1.7609e+00  7.7100e+01  1.1449e+01  2.0942e+01  3.0792e+01]
 [ 2.9348e+00  1.3696e+00  4.1955e+01  1.7445e+01  1.8859e+01  3.7050e+01]
 [ 2.5000e+00  3.2609e-01  2.5425e+01  2.8095e+01  1.8644e+01  3.3613e+01]
 [ 2.2826e+00  8.4783e-01  2.3716e+01  2.4625e+01  1.6897e+01  3.9794e+01]
 [ 4.2391e+00  6.5217e-02  7.1898e+01  2.4931e+01  2.4417e+01  1.5840e+01]
 [ 4.6739e+00  1.6304e+00  9.8015e+01  1.1460e+01  2.2235e+01  2.4804e+01]
 [ 1.4130e+00  5.8696e-01  9.3648e+00  3.2341e+01  1.1789e+01  4.8554e+01]
 [ 1.0870e-01  2.2826e+00  2.0888e+01  3.1309e+01 -4.1352e+00  8.2479e+01]
 [ 1.8478e+00  1.9565e-01  1.3811e+01  3.3018e+01  1.5026e+01  4.0361e+01]
 [ 2.7174e+00  7.1739e-01  3.1595e+01  2.3551e+01  1.9275e+01  3.4025e+01]
 [ 3.2609e-01  9.7826e-01

In [ ]:
iter_count = 150-num_init
current_iter = 0
X_step = X_init
Y_step = Y_init
C_step = C_init 

while current_iter < iter_count:
    Pareto = ParetoFront(Y_step)
    
    acquisition = AcquisitionHvEI(model_f,space,optimizer=aquisition_optimizer, jitter = 1e-3,
                                  model_c=model_c,jitter_c = np.array([0.0,0.0]),P=Pareto,r=np.array([140,60]))
    
    # --- CHOOSE a collection method
    evaluator = GPyOpt.core.evaluators.Sequential(acquisition)
    
    bo_step = ModularBayesianMultiobjectiveOptimization(model_f, space, objective, acquisition, evaluator, 
                                                        X_init = X_step, Y_init = Y_step, C_init = C_step, 
                                                        model_c = model_c, normalize_Y = False)
    
    x_next = bo_step.suggest_next_locations()
    y_next = binhKorn(x_next[0][0],x_next[0][1])
    c1_next = disc_constraint(x_next[0][0],x_next[0][1])
    c2_next = line_constraint(x_next[0][0],x_next[0][1])
    c_next = np.array([c1_next,c2_next])
    print("Current iteration - ",current_iter," : ",np.array([x_next[0][0],x_next[0][1],y_next[0],y_next[1],c_next[0],c_next[1]]))
    
    X_step = np.vstack((X_step, x_next))
    Y_step = np.vstack((Y_step, y_next))
    C_step = np.vstack((C_step, c_next))
    
    current_iter += 1

Current iteration -  0  :  [  4.9077   2.9681 131.5792   4.1372  16.182   37.4805]
Current iteration -  1  :  [  4.8592   2.9342 128.8848   4.2874  16.3707  37.3794]
Current iteration -  2  :  [  4.7973   2.9363 126.544    4.3     16.3371  37.7967]
Current iteration -  3  :  [  5.    3.  136.    4.   16.   37.3]
Current iteration -  4  :  [  4.922    2.9597 131.9457   4.1689  16.234   37.2919]
Current iteration -  5  :  [  4.7658   2.9914 126.6459   4.0894  15.9968  38.6565]
Current iteration -  6  :  [  4.7231   2.9838 124.8423   4.1419  16.0205  38.8434]
Current iteration -  7  :  [  4.921    2.9829 132.4556   4.0751  16.0962  37.5749]
Current iteration -  8  :  [  4.7602   2.9544 125.5521   4.2421  16.2143  38.2505]
Current iteration -  9  :  [  4.6123   2.9625 120.1967   4.3018  16.0734  39.328 ]
Current iteration -  10  :  [  4.6645   2.9601 122.0765   4.274   16.1255  38.9481]
Current iteration -  11  :  [  4.9557   2.992  134.0428   4.0341  16.046   37.4719]
Current iteration - 